In [1]:
library(TreePar)
library(TreeSim)
library(NELSI)
library(doParallel)
library(foreach)

Loading required package: ape
Loading required package: Matrix
Loading required package: subplex
Loading required package: TreeSim
Loading required package: laser
Loading required package: geiger
Loading required package: deSolve

Attaching package: ‘deSolve’

The following object is masked from ‘package:graphics’:

    matplot

Loading required package: foreach
Loading required package: iterators
Loading required package: parallel


## Diversification model adequacy

This is the most trivial example. I simulate a tree under constant speciation and extinction rates, refered to here as the 'reference' tree. I fit a rate shift model to estimate speciation and extinction rates for 0 rate shifts (i.e. constant rates). Using the speciation and extiction estimates from this model, I simulate a set of trees without rate shifts. For each simulated tree, I find the maximum likelihood under the model used to generate them (that with 0 rate shifts), resulting in a null distribution of the likelihood, under the 'correct' model. I calculate the *P* value of the likelihood of the 'reference' tree under the null likelihood of each of the models. I repeat this procedure 100 times and record each of the *P* values.

In [2]:
set.seed(1234)
nspecies <- 100
rho <- 0.5
lambda <- 3
mu <- 1.5

constant_rates_trees <- sim.bd.taxa(n = nspecies, numbsim = 100, lambda = lambda, mu = mu, frac = rho, complete = F)



#set.seed(10)
#nspecies <- 100
#time <- c(0, 0.5, 1) # At time 1 in the past, there is a rate shift
#rho <- c(0.5, 1, 1) #half of the present day species are sampled (rho[1] = 0.5)
#lambda <- c(1.5, 10, 5)# speciation rates, between t[i] and t[i+1] we have a speciation rate lambda
#mu <- c(1.5, 1.5, 1.5)# extinction rate. Similar notation as lambda

#Simulate a tree with a single rate shift
#constant_rates_trees <- sim.rateshift.taxa(nspecies, 1, lambda = lambda, mu = mu, frac = rho, times = time, complete = F)


In [3]:
fit_rate_shifts <- function(tree, rho){ # Rho at present.

    x_times <- sort(intnode.times(tree), decreasing = T)
    start <- min(x_times)
    end <- max(x_times)
    grid <- diff(range(x_times))
    res <- bd.shifts.optim(x_times, c(rho, 1, 1), grid, start, end, posdiv = T)[[2]]

    # Find likelihoods, lambda, mu, and rate-shift times
    likelihoods <- sapply(res, function(x) x[1]) 
    
    lambda0 <- res[[1]][3] / (1 - res[[1]][2])
    mu0 <- lambda0 * res[[1]][2]
   
    # The following are also computed, but note that some of them are negative and that this might
    # impede the tree simulation?
    lambda11 <- res[[2]][3] / (1 - res[[2]][2])
    mu11 <- lambda11 * res[[2]][2]
    lambda12 <- res[[2]][5] / (1 - res[[2]][4])
    mu12 <- lambda12 * res[[2]][4]
    time1 <- res[[2]][length(res[[2]])]
        
    lambda21 <- res[[3]][3] / (1 - res[[3]][2])
    mu21 <- lambda21 * res[[2]][2]
    lambda22 <- res[[3]][5] / (1 - res[[3]][4])
    mu22 <- lambda22 * res[[3]][4]
    lambda23 <- res[[3]][7] / (1 - res[[3]][6])
    mu23 <- lambda23 * res[[3]][6]
    time2 <- res[[3]][(length(res[[3]]) - 2):length(res[[3]])]
    
    return(list(likelihoods, shifts0= c(lambda0, mu0), shifts1=c(lambda11, lambda12, mu11, mu12, time1), 
               shifts2=c(lambda21, lambda22, lambda23, mu21, mu22, mu23, time2)))
}

In [4]:

cl <- makeCluster(4)
registerDoParallel(cl)

In [ ]:
pvals <- vector()

for(tr in 1:length(constant_rates_trees)){
    x_times <- sort(intnode.times(constant_rates_trees[[tr]]), decreasing = T)
    start <- min(x_times)
    end <- max(x_times)
    grid <- diff(range(x_times))
    
    reference_estimates <- fit_rate_shifts(constant_rates_trees[[tr]], 0.5)

    sim_trees0 <- sim.bd.taxa(n = nspecies, numbsim = 100, lambda = reference_estimates$shifts0[1], 
                              mu = reference_estimates$shifts0[2], frac = 0.5, complete = F)
    liks_sim_trees0 <- foreach(mt = sim_trees0, .packages = c('NELSI', 'TreePar')) %dopar% fit_rate_shifts(mt, 1)[[1]][1]
    pvals[tr] <- sum(reference_estimates[[1]][1] > liks_sim_trees0)
#    sim_trees1 <- sim.rateshift.taxa(nspecies, 10, lambda = reference_estimates$shifts1[1:2],
#                mu = reference_estimates$shifts1[3:4], frac = c(rho, 1), times = c(0, reference_estimates$shifts1[5]),
#                                     complete = F)
    
    
}

In [ ]:
stopCluster(cl)

In [ ]:
pdf('pvals_constant_constant.pdf', useDingbats = F)
hist(pvals)
dev.off()

In [ ]:
hist(pvals)